In [1]:
import json
import pandas as pd
from datetime import datetime as dt
from datetime import datetime, timedelta
import time
import csv
import statsmodels.api as sm
import os
import numpy as np
import math
import requests


In [4]:
df=pd.read_csv('Raw Data/pratham closing.csv')
df

,virtual_asset,price,time,trader,volume,fee,pnl,pnl_percent,position_side,liquidation,return,pnltocollateral,token_amount,price_into_token_amount
0,wETH,1333.300,2022-09-29 17:50:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,1.343000e+04,14.074640,248.630274,1.851305,LONG,No,0.218728,0.075,2.190287,2920.310029
1,wETH,1613.220,2023-01-29 01:22:27.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.812549e+04,38.125490,865.306194,2.269626,LONG,No,0.761238,0.075,2.897334,4674.036455
2,wETH,1646.990,2023-03-02 05:38:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.100001e+04,32.147008,192.458964,0.620835,SHORT,No,0.169312,0.075,0.719424,1184.883880
3,wETH,1089.900,2022-06-17 20:40:09.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,3.005672e+04,30.056718,175.465654,0.583782,SHORT,No,0.092935,0.041,1.353658,1475.352217
4,wETH,1269.530,2022-11-12 10:17:34.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,7.487485e+04,81.014588,-1252.061732,-1.672206,SHORT,No,-0.663153,0.041,1.830118,2323.389623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917,wBTC,30183.908,2023-04-11 13:55:35.000 UTC,0x7b076ddcb2abe2c9c167b7f4f57e18b5f5eb1c95,4.318619e+04,51.780242,-147.078592,-0.340569,SHORT,No,-0.412810,0.097,0.124933,3770.954976
9918,wBTC,17486.380,2023-01-10 20:21:21.000 UTC,0x10ee2ce2d6f0544ffa8222ca13878896e8124bad,2.862495e+05,326.324381,-4375.138125,-1.528435,SHORT,No,-11.383663,0.408,0.137138,2398.048264
9919,wETH,1871.830,2023-05-03 17:57:53.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,5.965176e+06,5965.175890,47898.179997,0.802963,SHORT,No,1.370841,0.265,114.555231,214427.918101
9920,wETH,1909.701,2023-05-03 22:17:24.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,1.479214e+07,15798.008144,100812.234636,0.681526,LONG,No,2.885236,0.265,201.647571,385086.568035


In [3]:
df['time'] = pd.to_datetime(df['time'])
df=df.sort_values(by='time')
df

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount
8433,wETH,3775.12000,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,LONG,0.114256,114.256339,0.006000,22.650720
7739,wETH,3300.49000,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,LONG,0.065694,65.693575,0.010000,33.004900
5544,wETH,3286.75517,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,LONG,1.506707,1506.706587,0.230000,755.953689
4379,wETH,3336.99000,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,0.667398,667.398000,0.100658,335.895396
14862,wETH,3336.99000,2021-09-12 06:15:07+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,1.996008,1996.007984,0.299641,999.899130
...,...,...,...,...,...,...,...,...,...
15364,wETH,1850.34400,2023-07-24 19:15:11+00:00,0x03b974abcc144061eef8ed8202caed448a4abe1a,SHORT,305.520619,293624.987684,10.808801,20000.000000
5574,wETH,1848.35400,2023-07-24 19:52:32+00:00,0x583a77c02c6c417dfb190c1056cdb286aa12c2c5,SHORT,145.631068,145631.067961,2.705110,5000.000000
3722,wETH,1848.44000,2023-07-25 00:57:17+00:00,0x5fc79d77b8c3e207cd62f7ab424a6ef5a03488ac,LONG,252.021901,252021.901430,13.503305,24960.049897
1075,UNI,5.80700,2023-07-25 02:38:23+00:00,0xe0c2e7b4267b1656add89520105faf63e66d4c05,SHORT,35.006817,32244.911964,172.205958,1000.000000
